In [106]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

# Loading data and cleaning table

In [108]:
concrete_data = pd.read_csv('concrete_data.csv') # I downloaded the file so I can use it locally
concrete_data.head() 

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [109]:
concrete_data.shape

(1030, 9)

In [110]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [111]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [112]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [113]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [114]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [115]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols # Cement, Blast, Ash, Water, Superplasti, Coarse, Fine, Age

8

# Building model

In [116]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(1))

In [117]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


### pip install pydot

In [23]:
keras.utils.plot_model(model)

In [118]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [119]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state = 42)

In [120]:
model.fit(X_train, Y_train, validation_data=(x_test, y_test), epochs=50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 1s 995us/step - loss: 1559.6955 - val_loss: 1469.3301
Epoch 2/50
721/721 [==============================] - 0s 107us/step - loss: 1540.3273 - val_loss: 1451.5919
Epoch 3/50
721/721 [==============================] - 0s 108us/step - loss: 1520.9699 - val_loss: 1433.4407
Epoch 4/50
721/721 [==============================] - 0s 93us/step - loss: 1500.7446 - val_loss: 1415.0774
Epoch 5/50
721/721 [==============================] - 0s 100us/step - loss: 1480.2884 - val_loss: 1395.9807
Epoch 6/50
721/721 [==============================] - 0s 90us/step - loss: 1458.9359 - val_loss: 1376.3320
Epoch 7/50
721/721 [==============================] - 0s 82us/step - loss: 1436.9276 - val_loss: 1356.0718
Epoch 8/50
721/721 [==============================] - 0s 96us/step - loss: 1414.2981 - val_loss: 1335.3853
Epoch 9/50
721/721 [==============================] - 0s 67us/step - loss: 1390.8290 - val_los

In [121]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: {}".format(scores))

Accuracy: 428.96918248050036


In [122]:
predicted = model.predict(x_test)
predicted[:5]

array([[51.166866],
       [43.28543 ],
       [61.985374],
       [42.989235],
       [ 8.188741]], dtype=float32)

In [123]:
true = []
for i in y_test:
    true.append(i)

In [124]:
mean_squared_error(true, predicted)

428.96918530861507

In [125]:
sc = []
mse = []
for i in range(51):
    X_train, x_test, Y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state = 42)
    model.fit(X_train, Y_train, validation_data=(x_test, y_test), epochs=50)
    scores = model.evaluate(x_test, y_test, verbose=0)
    sc.append(scores)
    predicted = model.predict(x_test)
    mse.append(mean_squared_error(true, predicted))

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 0s 71us/step - loss: 391.9074 - val_loss: 415.8355
Epoch 2/50
721/721 [==============================] - 0s 82us/step - loss: 379.3931 - val_loss: 402.8784
Epoch 3/50
721/721 [==============================] - 0s 64us/step - loss: 367.3123 - val_loss: 390.3824
Epoch 4/50
721/721 [==============================] - 0s 82us/step - loss: 355.7622 - val_loss: 378.2719
Epoch 5/50
721/721 [==============================] - 0s 64us/step - loss: 344.4825 - val_loss: 366.6538
Epoch 6/50
721/721 [==============================] - 0s 78us/step - loss: 333.8364 - val_loss: 354.9343
Epoch 7/50
721/721 [==============================] - 0s 76us/step - loss: 323.4348 - val_loss: 343.5612
Epoch 8/50
721/721 [==============================] - 0s 65us/step - loss: 313.2105 - val_loss: 333.1084
Epoch 9/50
721/721 [==============================] - 0s 82us/step - loss: 303.6568 - val_loss: 322.2088
Epoch 10/

In [130]:
mse[:5]

[147.42797030849323,
 116.17393936594216,
 85.63308873791406,
 66.73744988967546,
 56.53209368325012]

In [128]:
import statistics
dev = statistics.stdev(mse)  # Deviation of 50 mean squarred error, correspondin to 50 traning processes. 
dev

19.718929466284653